In [1]:
import os
%pwd

'/home/gautham/E2EProjects/TextSummarisation/research'

In [2]:
os.chdir("/home/gautham/E2EProjects/TextSummarisation")

In [3]:
! pwd

/home/gautham/E2EProjects/TextSummarisation


In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir : Path
    data_path : Path
    model_ckpt : str
    num_train_epochs : int
    warmup_steps : int
    per_device_train_batch_size : int
    weight_decay : float
    logging_steps : int
    evaluation_strategy : str
    eval_steps : int
    save_steps : float
    gradient_accumulation_steps : int

In [5]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories


class ConfigarationManager:
    def __init__(self, 
                 config_file_path = CONFIG_FILE_PATH,
                 param_file_path = PARAMS_FILE_FILE):
        

        self.config = read_yaml(config_file_path)
        self.params = read_yaml(param_file_path)

        create_directories([self.config.artifacts_root])
    
    def get_model_trainer_config(self)->ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_ckpt=config.model_ckpt,
            num_train_epochs=params.num_train_epochs,
            warmup_steps=params.warmup_steps,
            per_device_train_batch_size=params.per_device_train_batch_size,
            weight_decay=params.weight_decay,
            logging_steps=params.logging_steps,
            evaluation_strategy=params.evaluation_strategy,
            eval_steps=params.eval_steps,
            save_steps=1e6,
            gradient_accumulation_steps=params.gradient_accumulation_steps
        )

        return model_trainer_config

In [6]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from accelerate import DataLoaderConfiguration
from accelerate.data_loader import DataLoaderShard
from datasets import load_dataset, load_from_disk
import torch

class ModelTrainer:
    def __init__(self, config:ModelTrainerConfig) -> None:
        self.config = config

    def train(self):
        # device = "cuda" if torch.cuda.is_available() else "cpu"
        device = "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model_pegasus)

        # load data
        dataset_samsum_pt = load_from_disk(self.config.data_path)

        trainer_args = TrainingArguments(output_dir=self.config.root_dir,
                                         num_train_epochs=self.config.num_train_epochs,
                                         warmup_steps=self.config.warmup_steps,
                                         per_device_train_batch_size=self.config.per_device_train_batch_size,
                                         weight_decay=self.config.weight_decay,
                                         logging_steps=self.config.logging_steps,
                                         evaluation_strategy=self.config.evaluation_strategy,
                                         eval_steps=self.config.eval_steps,
                                         save_steps=self.config.save_steps,
                                         gradient_accumulation_steps=self.config.gradient_accumulation_steps)
        dataloader_config = DataLoaderConfiguration(dispatch_batches=None,
                                                    split_batches=False,
                                                    even_batches=True,
                                                    use_seedable_sampler=True)
        train_dataloader = DataLoaderShard(dataset_samsum_pt["test"], dataloader_config)
        eval_dataloader = DataLoaderShard(dataset_samsum_pt["validation"], dataloader_config)


        trainer = Trainer(model= model_pegasus, 
                          args=trainer_args,
                          train_dataset=train_dataloader,
                          eval_dataset=eval_dataloader)
        
        trainer.train()

        # save model
        model_pegasus.save_pretrained(os.path.join(self.config.root_dir, "pegasus-samsum-model"))
        tokenizer.save_pretrained(os.path.join(self.config.root_dir, "tokenizer"))

[2024-04-07 17:38:57,391: INFO: config: PyTorch version 2.2.1 available.]


In [7]:
# import torch
# torch.cuda.empty_cache()

In [8]:
try:
    config = ConfigarationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(model_trainer_config)
    model_trainer.train()
except Exception as exception:
    raise exception

[2024-04-07 17:38:57,789: INFO: common: yaml file : config/config.yaml loded successfully]
[2024-04-07 17:38:57,794: INFO: common: yaml file : params.yaml loded successfully]
[2024-04-07 17:38:57,796: INFO: common: Created directory at : artifacts]
[2024-04-07 17:38:57,798: INFO: common: Created directory at : artifacts/model_trainer]


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/gautham/miniconda3/envs/TextS/lib/python3.9/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


TypeError: 'DataLoaderShard' object is not subscriptable

In [1]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"

In [2]:
device

'cuda'

In [3]:
import torch
print(torch.version.cuda)

12.1
